In [2]:
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

csv_path = 'allData.csv'              # Path to your CSV
datetime_column = 'date'              # Adjust if needed
datetime_format = '%Y-%m-%d_%H:%M:%S' # Your datetime format

df = pd.read_csv(csv_path)
df[datetime_column] = pd.to_datetime(df[datetime_column], format=datetime_format)
start_time = df[datetime_column].iloc[0]
df['elapsed_sec'] = (df[datetime_column] - start_time).dt.total_seconds()

print("\nAvailable Columns:")
print(df.columns.tolist())


C:\Users\Hakima Lakhder\AppData\Local\Temp\ipykernel_35028\3645680377.py:11: DtypeWarning: Columns (30,31,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)



Available Columns:
['date', 'time', 'xi_rad', 'eta_rad', 'zeta_rad', 'xi_deg', 'eta_deg', 'zeta_deg', 'delta_T_norm', 'gear_up_flg', 'flaps_rad', 'IAS_kts', 'IAS_mDs', 'h_baro_ft', 'h_baro_m', 'h_pres_ft', 'h_pres_m', 'alpha_A_R_B_rad', 'beta_A_R_B_rad', 'alpha_A_R_B_deg', 'beta_A_R_B_deg', 'p_static_NDm2', 'OAT_degC', 'rho_kgDm3', 'h_dens_ft', 'h_dens_m', 'CAS_kts', 'CAS_mDs', 'TAS_kts', 'TAS_mDs', 'lat_WGS84_deg', 'lon_WGS84_deg', 'lat_WGS84_rad', 'lon_WGS84_rad', 'h_WHS84_dg', 'h_WGS84_m', 'Phi_deg', 'Theta_deg', 'Psi_deg', 'Phi_rad', 'Theta_rad', 'Psi_rad', 'p_degDs', 'q_degDs', 'r_degDs', 'p_radDs', 'q_radDs', 'r_radDs', 'n_x_B', 'n_y_B', 'n_z_B', 'u_K_R_E_O_ftDmin', 'v_K_R_E_O_ftDmin', 'w_K_R_E_O_ftDmin', 'u_K_R_E_O_mDs', 'v_K_R_E_O_mDs', 'w_K_R_E_O_mDs', 'V_K_R_E_abs_mDs', 'V_K_R_E_abs_kts', 'chi_K_R_deg', 'gamma_K_R_deg', 'chi_K_R_rad', 'gamma_K_R_rad', 'elapsed_sec']


In [14]:
import ipywidgets as widgets
from IPython.display import display

available_columns = [col for col in df.columns if df[col].dtype != 'object' and col != datetime_column]

multi_select = widgets.SelectMultiple(
    options=available_columns,
    description='Columns',
    rows=10,
    layout=widgets.Layout(width='50%')
)

confirm_button = widgets.Button(description="Confirm selection")
output = widgets.Output()

def on_confirm_clicked(b):
    with output:
        output.clear_output()
        selected = list(multi_select.value)
        print("You selected:", selected)
        # Here you can assign to a global variable or proceed with plotting, etc.
        global columns_to_plot
        columns_to_plot = selected

display(multi_select, confirm_button, output)
confirm_button.on_click(on_confirm_clicked)


SelectMultiple(description='Columns', layout=Layout(width='50%'), options=('time', 'xi_rad', 'eta_rad', 'zeta_…

Button(description='Confirm selection', style=ButtonStyle())

Output()

In [19]:
columns_to_plot = list(multi_select.value)
if not columns_to_plot:
    # If no columns selected, default to all numeric except datetime
    columns_to_plot = available_columns

print("Selected columns:", columns_to_plot)


Selected columns: ['h_baro_ft', 'Phi_deg', 'Theta_deg', 'Psi_deg']


In [20]:
filter_option = input("\nFilter by (1) elapsed seconds or (2) absolute datetime? (1/2/n): ").strip()

if filter_option == '1':
    t_start_input = input("Start time in seconds from start (or press Enter to skip): ").strip()
    t_end_input = input("End time in seconds from start (or press Enter to skip): ").strip()

    t_start = float(t_start_input) if t_start_input else df['elapsed_sec'].min()
    t_end = float(t_end_input) if t_end_input else df['elapsed_sec'].max()
    df = df[df['elapsed_sec'].between(t_start, t_end)]

elif filter_option == '2':
    dt_start_input = input("Start datetime (YYYY-MM-DD_HH:MM:SS) or press Enter to skip: ").strip()
    dt_end_input = input("End datetime (YYYY-MM-DD_HH:MM:SS) or press Enter to skip: ").strip()

    dt_start = datetime.strptime(dt_start_input, datetime_format) if dt_start_input else df[datetime_column].min()
    dt_end = datetime.strptime(dt_end_input, datetime_format) if dt_end_input else df[datetime_column].max()
    df = df[df[datetime_column].between(dt_start, dt_end)]

else:
    print("\nNo time filter selected — showing all rows.")



No time filter selected — showing all rows.


In [21]:
x_axis_choice = input("Use 'elapsed_sec' or 'date' as x-axis? (elapsed/date): ").strip().lower()
x_column = 'elapsed_sec' if x_axis_choice == 'elapsed' else datetime_column

grouping_series = df['elapsed_sec'].astype(int).rename('elapsed_sec_int')
df_downsampled = df.groupby(grouping_series).first().reset_index(drop=False)

if x_column == 'elapsed_sec':
    x_plot = 'elapsed_sec_int'
else:
    x_plot = datetime_column


In [ ]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure()

for col in columns_to_plot:
    y_raw = df_downsampled[col]
    y_min = y_raw.min()
    y_max = y_raw.max()
    
    if y_max == y_min:
        y_norm = np.zeros_like(y_raw)
    else:
        y_norm = (y_raw - y_min) / (y_max - y_min)
    
    fig.add_trace(go.Scatter(
        x=df_downsampled[x_plot],
        y=y_norm,
        mode='lines+markers',
        name=col,
        customdata=np.stack((y_raw,), axis=-1),
        hovertemplate='%{x}<br>' + col + ': %{customdata[0]:.3f}<extra></extra>'
    ))

fig.update_layout(
    title="Normalized Selected Data Over Time",
    xaxis_title=x_column,
    yaxis_title="Normalized Value (0 to 1)",
    hovermode='x unified',
    width=900,
    height=500,
    template='plotly_white'
)

fig.show()
